In [77]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder, OneHotEncoder

In [78]:
data = pd.read_excel('Terbaru.xlsx')
data=data.copy()

In [79]:
data

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Lat,Long
0,1,Mesjid Raya Baiturrahman,Mesjid bersejarah dengan arsitektur megah dan ...,ibadah,Banda Aceh,0,4.9,5.553774,95.317296
1,2,Kapal Apung Lampulo,Kapal yang terseret tsunami dan kini menjadi m...,alam,Banda Aceh,2000,4.6,5.546546,95.306845
2,3,Pulau Rubiah,Pulau yang terkenal dengan keindahan bawah lau...,alam,Sabang,0,4.9,5.882839,95.257308
3,4,Museum Tsunami,Museum yang menceritakan kejadian tsunami Aceh...,budaya,Banda Aceh,5000,4.6,5.547937,95.315265
4,5,Museum Rumah Cut Nyak Dien,Museum yang merupakan rumah pahlawan nasional ...,budaya,Aceh Besar,0,4.6,5.516920,95.271694
...,...,...,...,...,...,...,...,...,...
71,128,Teluk Kiluan,Teluk terkenal dengan atraksi lumba-lumba di L...,alam,Tangamus,0,4.6,-5.779489,105.093217
72,131,Pantai Tanjung Setia,Pantai dengan ombak besar untuk berselancar di...,alam,Pesisir Selatan,0,4.5,-5.302336,103.999850
73,132,Pantai Dewi Mandapa,Pantai dengan pemandangan indah dan fasilitas ...,alam,Pesarawan,0,3.9,-5.572197,105.243705
74,133,Bendungan Margotirto,Bendungan dengan pemandangan alam yang indah d...,alam,Tangamus,0,4.3,-5.421126,104.725779


In [80]:
# Extract features
features = ['Category', 'City', 'Rating', 'Price']
X = data[features].copy()

# Encode categorical features
le_category = LabelEncoder()
le_city = LabelEncoder()
X['Category'] = le_category.fit_transform(X['Category'])
X['City'] = le_city.fit_transform(X['City'])

# Scale numerical features
scaler = MinMaxScaler()
X[['Rating', 'Price']] = scaler.fit_transform(X[['Rating', 'Price']])


In [81]:
# Convert to numpy array
X = X.values

In [82]:
# Step 2: Feature Encoding
# One-hot encode Category and City
num_categories = len(le_category.classes_)
num_cities = len(le_city.classes_)

X_encoded = np.zeros((X.shape[0], num_categories + num_cities + 2))
X_encoded[:, :num_categories] = tf.keras.utils.to_categorical(X[:, 0], num_classes=num_categories)
X_encoded[:, num_categories:num_categories+num_cities] = tf.keras.utils.to_categorical(X[:, 1], num_classes=num_cities)
X_encoded[:, -2:] = X[:, 2:]  # Rating and Price

In [83]:
# Step 3: Model Architecture
input_dim = X_encoded.shape[1]
hidden_dim = 64

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(hidden_dim, activation='relu'),
    tf.keras.layers.Dense(input_dim, activation='sigmoid')
])

model.compile(optimizer='adam', loss='mse')


In [84]:
# Step 4: Training
# We'll use the same input as output for training (autoencoder-style)
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(X_encoded, test_size=0.2, random_state=42)

model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), verbose=1)

Epoch 1/50
2/2 [==============================] - 1s 403ms/step - loss: 0.2485 - val_loss: 0.2461
Epoch 2/50
2/2 [==============================] - 0s 67ms/step - loss: 0.2451 - val_loss: 0.2429
Epoch 3/50
2/2 [==============================] - 0s 72ms/step - loss: 0.2419 - val_loss: 0.2397
Epoch 4/50
2/2 [==============================] - 0s 78ms/step - loss: 0.2388 - val_loss: 0.2365
Epoch 5/50
2/2 [==============================] - 0s 87ms/step - loss: 0.2355 - val_loss: 0.2331
Epoch 6/50
2/2 [==============================] - 0s 118ms/step - loss: 0.2320 - val_loss: 0.2296
Epoch 7/50
2/2 [==============================] - 0s 88ms/step - loss: 0.2283 - val_loss: 0.2257
Epoch 8/50
2/2 [==============================] - 0s 70ms/step - loss: 0.2243 - val_loss: 0.2214
Epoch 9/50
2/2 [==============================] - 0s 65ms/step - loss: 0.2199 - val_loss: 0.2166
Epoch 10/50
2/2 [==============================] - 0s 70ms/step - loss: 0.2149 - val_loss: 0.2113
Epoch 11/50
2/2 [==========

In [85]:
model = tf.keras.models.load_model("model1.h5")

In [86]:
# Step 5: Inference
# Function to get feature vector for a given attraction
def get_feature_vector(attraction_name):
    attraction = data[data['Place_Name'] == attraction_name].iloc[0]
    features = [
        le_category.transform([attraction['Category']])[0],
        le_city.transform([attraction['City']])[0],
        attraction['Rating'],
        attraction['Price']
    ]
    
    encoded = np.zeros((1, input_dim))
    encoded[0, :num_categories] = tf.keras.utils.to_categorical(features[0], num_classes=num_categories)
    encoded[0, num_categories:num_categories+num_cities] = tf.keras.utils.to_categorical(features[1], num_classes=num_cities)
    encoded[0, -2:] = scaler.transform([features[2:]])

    
    return encoded

In [87]:
# Function to get recommendations
def get_recommendations(attraction_name, top_k=10):
    query_vector = get_feature_vector(attraction_name)
    query_embedding = model.predict(query_vector)
    
    all_embeddings = model.predict(X_encoded)
    similarities = np.dot(all_embeddings, query_embedding.T).flatten()
    
    top_indices = similarities.argsort()[::-1][1:top_k+1]  # Exclude the query itself
    top_attractions = data.iloc[top_indices]['Place_Name'].tolist()
    top_similarities = similarities[top_indices].tolist()

    
    return list(zip(top_attractions, top_similarities))

In [92]:
query_attraction = "Museum Rumah Cut Nyak Dien"  # Replace with an actual attraction from your dataset
recommendations = get_recommendations(query_attraction, top_k=10)

print(f"Top 5 recommendations similar to {query_attraction}:")
for attraction, similarity in recommendations:
    print(f"{attraction}: {similarity}")

1/1 [==============================] - 0s 33ms/step


C:\Users\User\anaconda3\envs\capstone\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


3/3 [==============================] - 0s 3ms/step
Top 5 recommendations similar to Museum Rumah Cut Nyak Dien:
Pulau Lengkuas: 0.8863500356674194
Pulau Batu Berlayar: 0.8846555948257446
Tanjung Tinggi: 0.8846555948257446
Tanjung Kelayang: 0.8828141093254089
Bukit Paramun: 0.8828141093254089
Danau Kaolin: 0.8801115155220032
Panorama Ampangan: 0.8655418157577515
Kepri Coral: 0.8606997132301331
Kebun Teh Dempo: 0.8595086336135864
Danau Laut Tawar: 0.8559842705726624
